In [ ]:
from sklearn.linear_model import SGDClassifier

from sklearn.kernel_approximation import RBFSampler, Nystroem

In [1]:
# Online SVM classifier is for training data in mini batches or strems.
class OnlineSVMClassifier():
    
    def __init__(self, loss='hinge', penalty='l2', alpha=0.0001, verbose=1):
        self.model = SGDClassifier(loss=loss, penalty=penalty, alpha=alpha, verbose=verbose)
    
    def partial_fit_realitime(self, X, Y, batch_size=None, max_itr = 100, W=None):
        
        old_val_score = 0 
        old_tr_score = 0 
        
        kf = KFold(n_splits=max_itr)
        X, Y = shuffle(X, Y)
        
        count = 0
        
        for train_index, test_index in kf.split(X):
            count = count + 1
            
            X_train, X_valid = X[train_index], X[test_index]
            Y_train, Y_valid = Y[train_index], Y[test_index]
            
            t = 0
            
            if batch_size is not None:
                pbar = tqdm(range(0, X_train.shape[0], batch_size))
                for i in pbar:
                    self.model.partial_fit(X_train[i:(i+batch_size)], Y_train[i:(i+batch_size)],classes=[0, 1] ,sample_weight=W)
                    t = t + self.score(X_train[i:(i+batch_size)], Y_train[i:(i+batch_size)])

                    pbar.set_description("Iteration %d" % count)

                tr_score = t/(X_train.shape[0]/batch_size)
            else:
                self.model.fit(X_train, Y_train ,sample_weight=W)
                tr_score = self.score(X_train, Y_train)
                
            val_score = self.score(X_valid, Y_valid)
            
            old_tr_score = tr_score
            old_val_score = val_score
            
            yield tr_score, val_score, count
        
    def partial_fit(self, X, Y, batch_size=None, max_itr = 100, W=None, anim=None):
        
        old_val_score = 0 
        old_tr_score = 0 
        
        kf = KFold(n_splits=max_itr)
        X, Y = shuffle(X, Y)
        
        counts = []
        count = 0
        
        training_scores = []
        validation_scores = []
        
        for train_index, test_index in kf.split(X):
            count = count + 1
            
            X_train, X_valid = X[train_index], X[test_index]
            Y_train, Y_valid = Y[train_index], Y[test_index]
            
            t = 0
            
            if batch_size is not None:
                pbar = tqdm(range(0, X_train.shape[0], batch_size))
                for i in pbar:
                    self.model.partial_fit(X_train[i:(i+batch_size)], Y_train[i:(i+batch_size)],classes=[0, 1] ,sample_weight=W)
                    t = t + self.score(X_train[i:(i+batch_size)], Y_train[i:(i+batch_size)])

                    pbar.set_description("Iteration %d" % count)

                tr_score = t/(X_train.shape[0]/batch_size)
            else:
                self.model.fit(X_train, Y_train ,sample_weight=W)
                tr_score = self.score(X_train, Y_train)
                
            val_score = self.score(X_valid, Y_valid)
            
            #             print('tr_score : ' , tr_score , ' val_score :',  val_score)
#             print('diff : ' , tr_score - val_score)
#             if tr_score < old_tr_score and val_score < old_val_score :
#                 break
            
            old_tr_score = tr_score
            old_val_score = val_score
            
            # Plotting
            counts.append(count)
            training_scores.append(tr_score)
            validation_scores.append(val_score)
            
            if anim is not None:
                anim.plotScores(np.vstack([np.array(counts), np.array(training_scores)]), np.vstack([np.array(counts), np.array(validation_scores)]))
                
        return training_scores, validation_scores
            
    def fit(self, X, Y, W=None):
        self.model.fit(X, Y, sample_weight=W)
        print('SVM training done-----')

    def predict(self, x):
        return self.model.predict(x)

    def score(self, x, y, w=None):
        score = self.model.score(x, y, sample_weight=w)
        return score
    
    def get_params(self, deep=True):
        return self.model.get_params(deep)

In [1]:
def Online_SVM_model(X_matrix, Y, x_matrix, y, kernel=None , batch_size = None , loss='hinge', penalty='l2', alpha=0.0001, max_itr = 100 ,verbose=1, plot=False, words=False, 
                     filename = None, animate=False):

    if kernel == 'rbf':
        print('RBF kernel...')
        X_features, x_features = RBFKernel(X_matrix, x_matrix)
    else:
        print('Linear kernel...')
        X_features, x_features = X_matrix, x_matrix
    
    print(X_features.shape, x_features.shape)
    online_svm = OnlineSVMClassifier(loss=loss, penalty=penalty, alpha=alpha, verbose=verbose)
    
    # Animate the plots
    anim = AnimateScores(max_itr)
    if animate:
        anim.plotScoresRealtime(frames = online_svm.partial_fit_realitime(X_features, Y, batch_size, max_itr = max_itr))
    else:
        training_scores, validation_scores = online_svm.partial_fit(X_features, Y, batch_size, max_itr = max_itr, anim=anim)
        anim.save(filename = filename + "-Animation-score")
        # Plot static score
        plot_scores(max_itr, training_scores, validation_scores, file=filename + "-score")
        
    
    if plot:
        plot_boundary(X_matrix, Y, svm, filename = filename, h=1)
    
    print('\n----------------------------Parameters---------------------------------------------\n')
    training_score = online_svm.score(X_features, Y)
    print('Trainign Accuracy score : ', training_score)
    print('\n------------------------------------------------------------------------------------\n')
          
    y_trainig_hat = online_svm.predict(X_features)
#     print('precision score ', precision_score(Y, y_trainig_hat))
#     print('\n------------------------------------------------------------------------------------\n')
#     print('f1 score ', f1_score(Y, y_trainig_hat))
    
    train_metric = metric_score(Y, y_trainig_hat, type='Training')

    test_score = online_svm.score(x_features, y)
    print('Test Accuracy score : ', test_score)
    print('\n------------------------------------------------------------------------------------\n')
          
    y_test_hat = online_svm.predict(x_features)
#     print('precision score ', precision_score(y, y_test_hat))
#     print('\n------------------------------------------------------------------------------------\n')
#     print('f1 score ', f1_score(y, y_test_hat))
    
    test_metric = metric_score(y, y_test_hat, type='Test')

    if filename is not None:
        pd_best = pd.DataFrame({"best_params" : {'model': online_svm.model, 'model__alpha': alpha}})
        save_results_html(filename, pd.concat([pd.concat([train_metric, test_metric], axis=1, sort=False), pd_best], keys=['Score', 'Model'], axis=1, sort=False))
    
    if words:
        print('------- Top words ---------')
        print_words(online_svm.model.coef_.T)

In [ ]:
def RBFKernel(X_matrix, x_matrix):
    
    m  = X_matrix.shape[1]
    
    rbf_feature = Nystroem(gamma=1/m , random_state=1, n_components=m)
    X_features = rbf_feature.fit_transform(X_matrix)
    
    x_features = rbf_feature.transform(x_matrix)
    
    return X_features, x_features